In [3]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda"


class Basic(nn.Module):
    def __init__(self, nhid = 64):
        super().__init__()
        self.nhid = nhid
        self.linear1 = nn.Linear(4, nhid)
        self.linear2 = nn.Linear(nhid, 2)
    
    def forward(self, src): #src is [batchsize * nroutes, npoints = 2, 2]
        src = src.view(-1, 4)
        hidden = F.relu(self.linear1(src))
        output = self.linear2(hidden)
        return output #[batchsize * nroutes, 2]


class Symmetric(nn.Module):
    def __init__(self, emsize = 64, nhid = 1024):
        super().__init__()
        self.nhid = nhid
        self.linear1 = nn.Linear(2, emsize)
        self.linear2 = nn.Linear(2, emsize)
        self.linear3 = nn.Linear(2 * emsize, nhid)
        self.linear4 = nn.Linear(nhid, 1)
        
    def forward(self, src): #src is [batchsize * nroutes, npoints = 2, 2]
        point1 = src[:, 0, :]
        point2 = src[:, 1, :]
        point1_em = self.linear1(point1)
        point2_em = self.linear1(point2)
        point1_mes = self.linear2(point1)
        point2_mes = self.linear2(point2)
        point1_agg = torch.cat([point1_em, point2_mes], dim = 1)
        point2_agg = torch.cat([point2_em, point1_mes], dim = 1)
        point1_hid = F.relu(self.linear3(point1_agg))
        point2_hid = F.relu(self.linear3(point2_agg))
        point1_out = self.linear4(point1_hid)
        point2_out = self.linear4(point2_hid)
        output = torch.cat([point1_out, point2_out], dim = 1)
        return output

NN = Symmetric().to(device)
optimizer = optim.Adam(NN.parameters())


def Calculate(src): #src is [batchsize * nroutes, npoints = 2, 2]
        
    circle1 = torch.cat([torch.tensor([[0,0], [2,0]], dtype = torch.float, device = device).unsqueeze(0).expand(src.size(0), 2, 2), src], dim = 1) #[batchsize * nroutes, 4, 2]
    circle2 = torch.cat([torch.tensor([[2,0], [0,2]], dtype = torch.float, device = device).unsqueeze(0).expand(src.size(0), 2, 2), src], dim = 1) #[batchsize * nroutes, 4, 2]
    circle3 = torch.cat([torch.tensor([[0,2], [0,0]], dtype = torch.float, device = device).unsqueeze(0).expand(src.size(0), 2, 2), src], dim = 1) #[batchsize * nroutes, 4, 2]
    incircle_matrix1 = torch.cat([circle1, (circle1 * circle1).sum(-1, keepdim = True), torch.ones([src.size(0), 4, 1], device = device)], dim = -1) #[batchsize * nroutes, 4, 4]
    incircle_matrix2 = torch.cat([circle2, (circle2 * circle2).sum(-1, keepdim = True), torch.ones([src.size(0), 4, 1], device = device)], dim = -1) #[batchsize * nroutes, 4, 4]
    incircle_matrix3 = torch.cat([circle3, (circle3 * circle3).sum(-1, keepdim = True), torch.ones([src.size(0), 4, 1], device = device)], dim = -1) #[batchsize * nroutes, 4, 4]
    incircle_test1 = incircle_matrix1.det() > 0 #[batchsize * nroutes]
    incircle_test2 = incircle_matrix2.det() > 0 #[batchsize * nroutes]
    incircle_test3 = incircle_matrix3.det() > 0 #[batchsize * nroutes]
    
    incircles = incircle_test1.float() + incircle_test2.float() + incircle_test3.float() #[batchsize * nroutes]
    output = torch.cat([(3 - incircles).unsqueeze(1), incircles.unsqueeze(1)], dim = 1) #[batchsize * nroutes, 2]
    return output #[batchsize * nroutes, 2]


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, points]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob += probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    
env = environment()


def train_calculate(epochs = 30000, npoints = 5, batchsize = 512, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        loss = (Calculate(env.points[:, 3:]) - NN(env.points[:, 3:])).square().mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        
        logits = NN(env.points[:, 3:])
        logits = torch.cat([torch.full([batchsize * nsamples, 3], float('-inf'), device = device), logits], dim = 1)
        next_point = env.sampleandgreedy_point(logits)
        logits = logits.scatter(1, next_point.unsqueeze(1), float('-inf')).squeeze(1)
        env.sampleandgreedy_point(logits)
        logprob = env.logprob
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        #baseline = 0.5 * torch.ones([1], device = device)
        baseline = greedy_baseline
        positive_reinforcement = -F.relu(-(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1)))
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        #optimizer.zero_grad()
        #loss.backward()
        #print(NN.encoder.weight.grad)
        #optimizer.step()
        #print(greedy_baseline.mean().item(), loss.item())
        print(greedy_baseline.mean().item())


def train(epochs = 30000, npoints = 5, batchsize = 512, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        logits = NN(env.points[:, 3:])
        logits = torch.cat([torch.full([batchsize * nsamples, 3], float('-inf'), device = device), logits], dim = 1)
        next_point = env.sampleandgreedy_point(logits)
        logits = logits.scatter(1, next_point.unsqueeze(1), float('-inf')).squeeze(1)
        env.sampleandgreedy_point(logits)
        logprob = env.logprob
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        #baseline = 0.5 * torch.ones([1], device = device)
        baseline = greedy_baseline
        positive_reinforcement = -F.relu(-(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1)))
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        """
        clipped reinforcement
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -0.5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item(), loss.item())
        print(greedy_baseline.mean().item())

In [ ]:
train_calculate()

0.529296875
0.486328125
0.474609375
0.501953125
0.52734375
0.46875
0.42578125
0.443359375
0.451171875
0.486328125
0.509765625
0.4609375
0.470703125
0.484375
0.37109375
0.328125
0.28515625
0.375
0.384765625
0.4296875
0.46484375
0.453125
0.4140625
0.4140625
0.37109375
0.306640625
0.228515625
0.2421875
0.248046875
0.3515625
0.32421875
0.37109375
0.376953125
0.36328125
0.30859375
0.271484375
0.21875
0.1953125
0.28515625
0.306640625
0.310546875
0.349609375
0.3125
0.310546875
0.32421875
0.21484375
0.1953125
0.2734375
0.326171875
0.330078125
0.322265625
0.29296875
0.3046875
0.240234375
0.212890625
0.240234375
0.2421875
0.271484375
0.26171875
0.251953125
0.2734375
0.240234375
0.2421875
0.20703125
0.205078125
0.228515625
0.244140625
0.22265625
0.2265625
0.21875
0.203125
0.2109375
0.25
0.212890625
0.23046875
0.234375
0.234375
0.23046875
0.216796875
0.240234375
0.205078125
0.236328125
0.220703125
0.2109375
0.224609375
0.21875
0.236328125
0.232421875
0.22265625
0.181640625
0.203125
0.224609375
0.2

0.1015625
0.078125
0.099609375
0.115234375
0.08203125
0.09375
0.08984375
0.11328125
0.09375
0.091796875
0.111328125
0.123046875
0.09375
0.1015625
0.107421875
0.078125
0.103515625
0.12109375
0.119140625
0.08984375
0.1171875
0.107421875
0.076171875
0.076171875
0.095703125
0.09375
0.10546875
0.09765625
0.0859375
0.095703125
0.07421875
0.08203125
0.095703125
0.083984375
0.09765625
0.08984375
0.0703125
0.09765625
0.087890625
0.072265625
0.08984375
0.095703125
0.126953125
0.095703125
0.099609375
0.09375
0.078125
0.091796875
0.091796875
0.10546875
0.095703125
0.115234375
0.1015625
0.072265625
0.115234375
0.109375
0.09765625
0.083984375
0.099609375
0.1015625
0.09375
0.080078125
0.09765625
0.08203125
0.09375
0.0859375
0.099609375
0.07421875
0.087890625
0.072265625
0.1015625
0.125
0.09765625
0.091796875
0.083984375
0.09765625
0.12890625
0.095703125
0.083984375
0.087890625
0.083984375
0.103515625
0.091796875
0.083984375
0.091796875
0.1015625
0.109375
0.107421875
0.09765625
0.099609375
0.087890625

0.07421875
0.087890625
0.087890625
0.07421875
0.103515625
0.0859375
0.08203125
0.0859375
0.068359375
0.08203125
0.1015625
0.0859375
0.091796875
0.0859375
0.078125
0.080078125
0.064453125
0.05859375
0.08984375
0.072265625
0.064453125
0.08203125
0.078125
0.087890625
0.0703125
0.060546875
0.08984375
0.080078125
0.076171875
0.091796875
0.07421875
0.072265625
0.064453125
0.08203125
0.07421875
0.080078125
0.080078125
0.06640625
0.06640625
0.072265625
0.076171875
0.0546875
0.078125
0.064453125
0.0703125
0.08984375
0.072265625
0.080078125
0.0859375
0.06640625
0.056640625
0.07421875
0.0859375
0.064453125
0.099609375
0.08203125
0.08203125
0.07421875
0.08203125
0.07421875
0.068359375
0.08203125
0.07421875
0.08203125
0.08203125
0.0859375
0.07421875
0.068359375
0.056640625
0.072265625
0.083984375
0.068359375
0.080078125
0.080078125
0.064453125
0.080078125
0.072265625
0.107421875
0.0859375
0.06640625
0.09765625
0.0859375
0.078125
0.0703125
0.068359375
0.056640625
0.0703125
0.06640625
0.076171875
0.0

0.0703125
0.044921875
0.05859375
0.05859375
0.0625
0.083984375
0.060546875
0.068359375
0.0703125
0.0859375
0.08203125
0.0859375
0.05859375
0.068359375
0.076171875
0.0703125
0.048828125
0.0703125
0.072265625
0.076171875
0.064453125
0.078125
0.0625
0.078125
0.064453125
0.08984375
0.076171875
0.052734375
0.07421875
0.06640625
0.06640625
0.07421875
0.08203125
0.0859375
0.068359375
0.080078125
0.083984375
0.05078125
0.07421875
0.060546875
0.0625
0.064453125
0.0703125
0.068359375
0.0625
0.06640625
0.0625
0.0703125
0.05859375
0.060546875
0.076171875
0.0625
0.09375
0.0859375
0.087890625
0.091796875
0.087890625
0.0703125
0.0703125
0.072265625
0.072265625
0.078125
0.0625
0.056640625
0.072265625
0.060546875
0.0703125
0.07421875
0.068359375
0.05078125
0.0859375
0.0703125
0.06640625
0.07421875
0.05859375
0.08203125
0.0703125
0.060546875
0.05859375
0.05859375
0.07421875
0.0703125
0.078125
0.064453125
0.064453125
0.080078125
0.06640625
0.0703125
0.0703125
0.072265625
0.080078125
0.083984375
0.0683593

0.052734375
0.078125
0.05859375
0.0625
0.0546875
0.0625
0.05859375
0.064453125
0.046875
0.044921875
0.064453125
0.080078125
0.080078125
0.056640625
0.06640625
0.080078125
0.07421875
0.068359375
0.076171875
0.0703125
0.087890625
0.072265625
0.060546875
0.078125
0.048828125
0.0703125
0.078125
0.068359375
0.056640625
0.08984375
0.078125
0.068359375
0.060546875
0.06640625
0.06640625
0.06640625
0.07421875
0.076171875
0.068359375
0.06640625
0.07421875
0.0703125
0.06640625
0.052734375
0.068359375
0.07421875
0.06640625
0.064453125
0.078125
0.068359375
0.037109375
0.083984375
0.0703125
0.060546875
0.078125
0.08203125
0.07421875
0.080078125
0.05859375
0.08203125
0.064453125
0.068359375
0.05078125
0.060546875
0.060546875
0.078125
0.060546875
0.076171875
0.05859375
0.072265625
0.0703125
0.076171875
0.068359375
0.07421875
0.083984375
0.078125
0.052734375
0.068359375
0.06640625
0.064453125
0.08203125
0.05078125
0.078125
0.060546875
0.06640625
0.07421875
0.05078125
0.06640625
0.068359375
0.068359375


0.052734375
0.046875
0.068359375
0.078125
0.078125
0.0703125
0.083984375
0.068359375
0.06640625
0.0625
0.068359375
0.060546875
0.06640625
0.05859375
0.072265625
0.080078125
0.0625
0.056640625
0.05859375
0.0625
0.06640625
0.060546875
0.07421875
0.0703125
0.0703125
0.068359375
0.044921875
0.060546875
0.056640625
0.0625
0.060546875
0.0390625
0.056640625
0.08203125
0.05859375
0.0703125
0.0625
0.0625
0.0546875
0.064453125
0.07421875
0.068359375
0.095703125
0.072265625
0.07421875
0.076171875
0.078125
0.0703125
0.041015625
0.05859375
0.078125
0.0625
0.0625
0.064453125
0.0625
0.078125
0.08203125
0.046875
0.060546875
0.052734375
0.0546875
0.056640625
0.052734375
0.056640625
0.05859375
0.064453125
0.080078125
0.0703125
0.0703125
0.064453125
0.0703125
0.08203125
0.060546875
0.056640625
0.0625
0.048828125
0.0625
0.07421875
0.06640625
0.07421875
0.0625
0.0703125
0.078125
0.0625
0.056640625
0.060546875
0.0546875
0.0625
0.05859375
0.08203125
0.07421875
0.05078125
0.072265625
0.068359375
0.09375
0.066

0.05859375
0.05859375
0.06640625
0.068359375
0.08203125
0.0703125
0.07421875
0.060546875
0.048828125
0.07421875
0.068359375
0.056640625
0.0546875
0.05859375
0.025390625
0.05859375
0.05859375
0.068359375
0.076171875
0.060546875
0.078125
0.060546875
0.0703125
0.046875
0.068359375
0.041015625
0.0546875
0.060546875
0.052734375
0.05859375
0.06640625
0.0703125
0.05078125
0.05859375
0.048828125
0.037109375
0.07421875
0.05859375
0.060546875
0.037109375
0.06640625
0.052734375
0.08984375
0.072265625
0.05078125
0.060546875
0.046875
0.0859375
0.06640625
0.060546875
0.0546875
0.07421875
0.05859375
0.0625
0.060546875
0.080078125
0.064453125
0.05859375
0.052734375
0.046875
0.06640625
0.0625
0.064453125
0.0546875
0.056640625
0.068359375
0.060546875
0.052734375
0.0390625
0.056640625
0.05859375
0.080078125
0.068359375
0.072265625
0.05859375
0.068359375
0.064453125
0.05859375
0.07421875
0.048828125
0.048828125
0.0703125
0.056640625
0.05078125
0.0859375
0.072265625
0.0859375
0.068359375
0.0546875
0.058593

0.044921875
0.068359375
0.0546875
0.052734375
0.03515625
0.076171875
0.06640625
0.0703125
0.0546875
0.0546875
0.056640625
0.072265625
0.033203125
0.0703125
0.064453125
0.060546875
0.072265625
0.060546875
0.046875
0.0625
0.048828125
0.05859375
0.078125
0.0625
0.0625
0.0546875
0.068359375
0.083984375
0.060546875
0.05859375
0.068359375
0.060546875
0.060546875
0.048828125
0.0625
0.0625
0.0546875
0.080078125
0.072265625
0.0625
0.037109375
0.060546875
0.05078125
0.0625
0.078125
0.05859375
0.05859375
0.03515625
0.060546875
0.0625
0.0625
0.0703125
0.052734375
0.06640625
0.06640625
0.0625
0.052734375
0.046875
0.05078125
0.072265625
0.046875
0.087890625
0.064453125
0.05078125
0.05859375
0.060546875
0.06640625
0.052734375
0.048828125
0.07421875
0.056640625
0.056640625
0.05078125
0.05078125
0.044921875
0.064453125
0.056640625
0.056640625
0.076171875
0.05859375
0.0546875
0.0546875
0.060546875
0.07421875
0.064453125
0.064453125
0.064453125
0.05859375
0.080078125
0.04296875
0.052734375
0.05078125
0.0

0.060546875
0.060546875
0.03515625
0.052734375
0.06640625
0.0390625
0.05859375
0.056640625
0.0546875
0.052734375
0.056640625
0.05859375
0.0703125
0.0625
0.052734375
0.080078125
0.05078125
0.0390625
0.05078125
0.029296875
0.052734375
0.052734375
0.0625
0.052734375
0.052734375
0.052734375
0.0703125
0.0390625
0.048828125
0.037109375
0.064453125
0.04296875
0.0625
0.04296875
0.064453125
0.0546875
0.05078125
0.03515625
0.068359375
0.052734375
0.064453125
0.046875
0.0390625
0.05859375
0.04296875
0.048828125
0.056640625
0.0546875
0.076171875
0.07421875
0.0546875
0.052734375
0.060546875
0.06640625
0.05078125
0.060546875
0.0703125
0.0625
0.04296875
0.056640625
0.044921875
0.0546875
0.06640625
0.072265625
0.05078125
0.05078125
0.056640625
0.060546875
0.064453125
0.056640625
0.05859375
0.05078125
0.06640625
0.064453125
0.064453125
0.048828125
0.05078125
0.048828125
0.05859375
0.05078125
0.041015625
0.05078125
0.0625
0.064453125
0.05078125
0.0703125
0.04296875
0.0703125
0.05078125
0.05859375
0.0429

0.05859375
0.05859375
0.0546875
0.056640625
0.072265625
0.064453125
0.037109375
0.048828125
0.052734375
0.056640625
0.052734375
0.05078125
0.048828125
0.0625
0.04296875
0.04296875
0.060546875
0.0546875
0.064453125
0.0390625
0.06640625
0.060546875
0.046875
0.0625
0.048828125
0.06640625
0.0625
0.0390625
0.064453125
0.048828125
0.033203125
0.056640625
0.05078125
0.078125
0.064453125
0.056640625
0.056640625
0.044921875
0.052734375
0.05078125
0.046875
0.05859375
0.052734375
0.04296875
0.05859375
0.05859375
0.064453125
0.060546875
0.048828125
0.05078125
0.06640625
0.041015625
0.0390625
0.0625
0.05859375
0.0546875
0.05078125
0.05078125
0.048828125
0.0546875
0.056640625


In [ ]:
train() #HEAVILY CLIPPED LOSS, nhid = 1024

0.513671875
0.48046875
0.521484375
0.34765625
0.521484375
0.48046875
0.474609375
0.482421875
0.5078125
0.486328125
0.298828125
0.421875
0.4765625
0.466796875
0.32421875
0.37109375
0.51171875
0.45703125
0.26953125
0.40234375
0.44140625
0.40625
0.271484375
0.525390625
0.478515625
0.451171875
0.49609375
0.330078125
0.5
0.4765625
0.33203125
0.470703125
0.259765625
0.431640625
0.310546875
0.419921875
0.361328125
0.30078125
0.447265625
0.248046875
0.294921875
0.279296875
0.267578125
0.3203125
0.2578125
0.330078125
0.447265625
0.330078125
0.44921875
0.3671875
0.427734375
0.3984375
0.298828125
0.318359375
0.26953125
0.3046875
0.236328125
0.33984375
0.248046875
0.33984375
0.296875
0.41015625
0.38671875
0.349609375
0.42578125
0.30078125
0.337890625
0.2421875
0.248046875
0.244140625
0.267578125
0.27734375
0.2265625
0.2890625
0.205078125
0.322265625
0.23046875
0.326171875
0.322265625
0.369140625
0.357421875
0.27734375
0.298828125
0.19921875
0.259765625
0.234375
0.26953125
0.173828125
0.267578125
0

0.19921875
0.12109375
0.095703125
0.087890625
0.1640625
0.208984375
0.111328125
0.060546875
0.169921875
0.197265625
0.068359375
0.201171875
0.2109375
0.06640625
0.1015625
0.150390625
0.1484375
0.09765625
0.095703125
0.1171875
0.115234375
0.169921875
0.12890625
0.056640625
0.0703125
0.228515625
0.115234375
0.07421875
0.201171875
0.193359375
0.1171875
0.1015625
0.099609375
0.154296875
0.158203125
0.11328125
0.119140625
0.130859375
0.1171875
0.119140625
0.154296875
0.1796875
0.1328125
0.123046875
0.12890625
0.171875
0.169921875
0.11328125
0.083984375
0.111328125
0.146484375
0.134765625
0.103515625
0.080078125
0.083984375
0.07421875
0.09375
0.080078125
0.142578125
0.17578125
0.08984375
0.083984375
0.138671875
0.14453125
0.103515625
0.111328125
0.126953125
0.125
0.107421875
0.119140625
0.185546875
0.173828125
0.125
0.12109375
0.1171875
0.12890625
0.18359375
0.109375
0.115234375
0.23046875
0.072265625
0.109375
0.2265625
0.087890625
0.236328125
0.130859375
0.130859375
0.24609375
0.11328125
0.

0.0859375
0.091796875
0.076171875
0.076171875
0.0859375
0.091796875
0.091796875
0.0546875
0.05078125
0.0625
0.109375
0.126953125
0.078125
0.060546875
0.06640625
0.111328125
0.08984375
0.046875
0.185546875
0.07421875
0.0625
0.1796875
0.078125
0.08203125
0.197265625
0.0625
0.1328125
0.10546875
0.07421875
0.16015625
0.083984375
0.083984375
0.171875
0.044921875
0.123046875
0.076171875
0.0546875
0.0859375
0.095703125
0.046875
0.060546875
0.05859375
0.064453125
0.052734375
0.072265625
0.080078125
0.08984375
0.09765625
0.0625
0.046875
0.052734375
0.0859375
0.193359375
0.07421875
0.0546875
0.201171875
0.0703125
0.1796875
0.046875
0.109375
0.158203125
0.072265625
0.052734375
0.119140625
0.068359375
0.044921875
0.091796875
0.09375
0.068359375
0.056640625
0.052734375
0.123046875
0.130859375
0.076171875
0.06640625
0.080078125
0.076171875
0.115234375
0.126953125
0.08203125
0.05859375
0.076171875
0.10546875
0.103515625
0.076171875
0.11328125
0.146484375
0.060546875
0.0859375
0.1484375
0.07421875
0.0

0.091796875
0.234375
0.154296875
0.1953125
0.162109375
0.185546875
0.123046875
0.12890625
0.115234375
0.150390625
0.080078125
0.166015625
0.07421875
0.212890625
0.1171875
0.201171875
0.16015625
0.171875
0.150390625
0.140625
0.08984375
0.12109375
0.076171875
0.150390625
0.083984375
0.166015625
0.10546875
0.166015625
0.154296875
0.16796875
0.154296875
0.0859375
0.16796875
0.087890625
0.083984375
0.12109375
0.060546875
0.09765625
0.080078125
0.158203125
0.048828125
0.12890625
0.06640625
0.171875
0.095703125
0.166015625
0.177734375
0.16015625
0.14453125
0.0703125
0.119140625
0.056640625
0.193359375
0.052734375
0.130859375
0.080078125
0.1640625
0.078125
0.154296875
0.123046875
0.146484375
0.18359375
0.103515625
0.138671875
0.05859375
0.103515625
0.076171875
0.06640625
0.140625
0.0625
0.142578125
0.09375
0.15625
0.109375
0.10546875
0.115234375
0.099609375
0.087890625
0.134765625
0.07421875
0.1484375
0.09765625
0.134765625
0.171875
0.078125
0.185546875
0.06640625
0.140625
0.05078125
0.1601562

0.046875
0.072265625
0.095703125
0.044921875
0.037109375
0.05859375
0.078125
0.0390625
0.056640625
0.0859375
0.05078125
0.072265625
0.056640625
0.05078125
0.09765625
0.048828125
0.052734375
0.037109375
0.09765625
0.087890625
0.046875
0.072265625
0.07421875
0.060546875
0.06640625
0.056640625
0.166015625
0.0546875
0.07421875
0.0625
0.044921875
0.044921875
0.06640625
0.0625
0.046875
0.09375
0.046875
0.076171875
0.06640625
0.064453125
0.099609375
0.03515625
0.056640625
0.068359375
0.0859375
0.14453125
0.056640625
0.08984375
0.08984375
0.05078125
0.056640625
0.0703125
0.056640625
0.072265625
0.087890625
0.03515625
0.072265625
0.080078125
0.076171875
0.064453125
0.0625
0.060546875
0.0546875
0.068359375
0.046875
0.041015625
0.0234375
0.044921875
0.0390625
0.029296875
0.046875
0.06640625
0.05859375
0.05859375
0.07421875
0.033203125
0.052734375
0.095703125
0.0625
0.041015625
0.076171875
0.05859375
0.03125
0.1015625
0.05859375
0.072265625
0.080078125
0.03125
0.037109375
0.05078125
0.083984375
0.

In [3]:
train() #HEAVILY CLIPPED LOSS, nhid = 64

0.466796875
0.513671875
0.484375
0.486328125
0.50390625
0.4921875
0.53125
0.44921875
0.3984375
0.416015625
0.380859375
0.41796875
0.3671875
0.37890625
0.427734375
0.451171875
0.431640625
0.435546875
0.451171875
0.462890625
0.52734375
0.478515625
0.509765625
0.458984375
0.4609375
0.451171875
0.41015625
0.396484375
0.41015625
0.369140625
0.37109375
0.39453125
0.376953125
0.380859375
0.353515625
0.302734375
0.337890625
0.38671875
0.423828125
0.41796875
0.416015625
0.41015625
0.361328125
0.3828125
0.310546875
0.369140625
0.34765625
0.3203125
0.310546875
0.3671875
0.375
0.40234375
0.392578125
0.40234375
0.380859375
0.419921875
0.369140625
0.388671875
0.390625
0.400390625
0.376953125
0.416015625
0.388671875
0.41015625
0.41796875
0.369140625
0.404296875
0.3984375
0.376953125
0.376953125
0.384765625
0.42578125
0.3515625
0.39453125
0.3828125
0.38671875
0.41015625
0.33984375
0.37109375
0.322265625
0.369140625
0.32421875
0.349609375
0.337890625
0.302734375
0.359375
0.34765625
0.322265625
0.357421

0.2421875
0.244140625
0.23046875
0.279296875
0.208984375
0.26171875
0.236328125
0.208984375
0.25390625
0.236328125
0.201171875
0.2421875
0.271484375
0.212890625
0.2578125
0.291015625
0.271484375
0.255859375
0.25390625
0.2109375
0.232421875
0.28125
0.322265625
0.212890625
0.2734375
0.279296875
0.275390625
0.20703125
0.265625
0.265625
0.21875
0.25
0.193359375
0.244140625
0.2734375
0.232421875
0.26171875
0.220703125
0.23046875
0.248046875
0.228515625
0.287109375
0.279296875
0.2109375
0.2109375
0.271484375
0.205078125
0.2265625
0.28125
0.22265625
0.251953125
0.248046875
0.203125
0.259765625
0.26953125
0.1875
0.216796875
0.23046875
0.18359375
0.212890625
0.25390625
0.25390625
0.205078125
0.224609375
0.234375
0.248046875
0.228515625
0.216796875
0.23046875
0.234375
0.224609375
0.232421875
0.19921875
0.240234375
0.255859375
0.212890625
0.236328125
0.2578125
0.1953125
0.255859375
0.26171875
0.25390625
0.18359375
0.208984375
0.26953125
0.21875
0.21875
0.275390625
0.224609375
0.232421875
0.236328

0.16015625
0.23828125
0.23046875
0.23046875
0.19921875
0.220703125
0.193359375
0.1953125
0.17578125
0.1953125
0.21484375
0.220703125
0.28125
0.22265625
0.20703125
0.2109375
0.28515625
0.234375
0.19140625
0.234375
0.201171875
0.1953125
0.212890625
0.22265625
0.263671875
0.232421875
0.203125
0.123046875
0.17578125
0.1875
0.22265625
0.185546875
0.26953125
0.216796875
0.216796875
0.203125
0.23828125
0.2265625
0.21875
0.1640625
0.181640625
0.166015625
0.22265625
0.205078125
0.16015625
0.205078125
0.173828125
0.212890625
0.21875
0.1796875
0.18359375
0.17578125
0.197265625
0.16796875
0.201171875
0.232421875
0.220703125
0.189453125
0.193359375
0.236328125
0.185546875
0.1953125
0.24609375
0.24609375
0.21484375
0.220703125
0.19140625
0.173828125
0.19921875
0.2109375
0.203125
0.265625
0.177734375
0.197265625
0.201171875
0.201171875
0.220703125
0.216796875
0.1796875
0.15625
0.1953125
0.224609375
0.154296875
0.234375
0.236328125
0.166015625
0.158203125
0.2578125
0.2421875
0.162109375
0.193359375
0.

0.291015625
0.181640625
0.228515625
0.296875
0.171875
0.3359375
0.205078125
0.365234375
0.3046875
0.341796875
0.373046875
0.23046875
0.296875
0.205078125
0.244140625
0.21875
0.181640625
0.287109375
0.177734375
0.220703125
0.220703125
0.171875
0.189453125
0.2578125
0.169921875
0.298828125
0.20703125
0.185546875
0.263671875
0.15625
0.29296875
0.169921875
0.3359375
0.173828125
0.244140625
0.201171875
0.212890625
0.2109375
0.212890625
0.310546875
0.1875
0.3671875
0.236328125
0.375
0.396484375
0.267578125
0.296875
0.1953125
0.32421875
0.25
0.296875
0.30078125
0.271484375
0.30078125
0.30078125
0.234375
0.369140625
0.248046875
0.28125
0.326171875
0.30078125
0.31640625
0.232421875
0.21484375
0.28125
0.19921875
0.24609375
0.19921875
0.3125
0.2265625
0.26953125
0.302734375
0.2265625
0.328125
0.23046875
0.23828125
0.19921875
0.19921875
0.2265625
0.19140625
0.279296875
0.203125
0.31640625
0.28125
0.283203125
0.291015625
0.197265625
0.267578125
0.21875
0.2265625
0.23828125
0.19140625
0.27734375
0.1

0.265625
0.15625
0.19921875
0.24609375
0.2109375
0.15625
0.1640625
0.234375
0.2265625
0.177734375
0.189453125
0.19140625
0.1796875
0.1953125
0.1796875
0.21875
0.228515625
0.171875
0.19140625
0.259765625
0.216796875
0.154296875
0.16796875
0.21484375
0.197265625
0.1328125
0.26171875
0.18359375
0.162109375
0.2109375
0.234375
0.173828125
0.185546875
0.15234375
0.171875
0.19921875
0.23046875
0.212890625
0.173828125
0.25390625
0.26171875
0.154296875
0.220703125
0.228515625
0.212890625
0.181640625
0.216796875
0.22265625
0.115234375
0.162109375
0.1796875
0.197265625
0.173828125
0.140625
0.177734375
0.279296875
0.1640625
0.1484375
0.291015625
0.1875
0.15234375
0.279296875
0.169921875
0.177734375
0.26171875
0.134765625
0.173828125
0.236328125
0.181640625
0.208984375
0.1640625
0.23046875
0.24609375
0.244140625
0.17578125
0.173828125
0.185546875
0.18359375
0.16015625
0.171875
0.154296875
0.142578125
0.13671875
0.203125
0.189453125
0.140625
0.15234375
0.28125
0.205078125
0.150390625
0.302734375
0.2

...

0.14453125
0.125
0.109375
0.111328125
0.115234375
0.078125
0.140625
0.103515625
0.138671875
0.091796875
0.08984375
0.095703125
0.099609375
0.09375
0.068359375
0.1328125
0.05859375
0.09375
0.08984375
0.119140625
0.0859375
0.09765625
0.068359375
0.076171875
0.080078125
0.056640625
0.076171875
0.0625
0.09375
0.048828125
0.111328125
0.08984375
0.087890625
0.099609375
0.087890625
0.162109375
0.08203125
0.140625
0.076171875
0.140625
0.08984375
0.15234375
0.146484375
0.1640625
0.154296875
0.12109375
0.130859375
0.078125
0.12890625
0.068359375
0.103515625
0.083984375
0.076171875
0.0859375
0.072265625
0.103515625
0.083984375
0.162109375
0.1015625
0.16015625
0.16796875
0.126953125
0.17578125
0.103515625
0.09765625
0.080078125
0.087890625
0.068359375
0.0625
0.0703125
0.083984375
0.146484375
0.064453125
0.12109375
0.0625
0.111328125
0.08203125
0.1484375
0.08203125
0.150390625
0.064453125
0.08984375
0.0625
0.09375
0.078125
0.083984375
0.10546875
0.1015625
0.103515625
0.1328125
0.12890625
0.14257812

0.177734375
0.087890625
0.1328125
0.0546875
0.125
0.08984375
0.11328125
0.05078125
0.1171875
0.123046875
0.056640625
0.103515625
0.060546875
0.13671875
0.0546875
0.107421875
0.048828125
0.080078125
0.048828125
0.10546875
0.083984375
0.056640625
0.08203125
0.091796875
0.046875
0.078125
0.07421875
0.056640625
0.099609375
0.103515625
0.1328125
0.056640625
0.123046875
0.08203125
0.140625
0.11328125
0.12890625
0.181640625
0.123046875
0.1328125
0.0859375
0.150390625
0.041015625
0.123046875
0.076171875
0.06640625
0.078125
0.0546875
0.064453125
0.068359375
0.06640625
0.107421875
0.07421875
0.072265625
0.068359375
0.06640625
0.0703125
0.076171875
0.08984375
0.08984375
0.09375
0.091796875
0.0546875
0.095703125
0.095703125
0.06640625
0.134765625
0.07421875
0.15625
0.08203125
0.173828125
0.103515625
0.1953125
0.140625
0.142578125
0.12109375
0.068359375
0.10546875
0.06640625
0.091796875
0.107421875
0.0546875
0.1328125
0.1171875
0.08203125
0.10546875
0.109375
0.140625
0.087890625
0.212890625
0.08007

0.072265625
0.060546875
0.115234375
0.072265625
0.041015625
0.064453125
0.05859375
0.06640625
0.08203125
0.068359375
0.107421875
0.060546875
0.056640625
0.12109375
0.095703125
0.078125
0.1015625
0.064453125
0.060546875
0.083984375
0.07421875
0.126953125
0.095703125
0.15234375
0.048828125
0.134765625
0.064453125
0.126953125
0.09375
0.052734375
0.11328125
0.068359375
0.0703125
0.060546875
0.072265625
0.09375
0.0703125
0.048828125
0.068359375
0.06640625
0.064453125
0.076171875
0.056640625
0.109375
0.083984375
0.06640625
0.095703125
0.076171875
0.09375
0.0859375
0.091796875
0.10546875
0.056640625
0.109375
0.091796875
0.060546875
0.109375
0.064453125
0.1015625
0.064453125
0.078125
0.076171875
0.076171875
0.060546875
0.05859375
0.064453125
0.080078125
0.064453125
0.052734375
0.095703125
0.091796875
0.0625
0.111328125
0.087890625
0.142578125
0.064453125
0.130859375
0.060546875
0.0859375
0.095703125
0.04296875
0.056640625
0.080078125
0.07421875
0.060546875
0.06640625
0.091796875
0.068359375
0.

0.189453125
0.12109375
0.2109375
0.16796875
0.17578125
0.201171875
0.12890625
0.134765625
0.1015625
0.072265625
0.125
0.044921875
0.09375
0.048828125
0.130859375
0.06640625
0.109375
0.0703125
0.1171875
0.0625
0.08203125
0.08203125
0.076171875
0.146484375
0.056640625
0.109375
0.0625
0.13671875
0.103515625
0.0703125
0.099609375
0.076171875
0.177734375
0.080078125
0.169921875
0.08203125
0.173828125
0.134765625
0.1640625
0.177734375
0.15234375
0.16015625
0.123046875
0.09765625
0.11328125
0.083984375
0.125
0.072265625
0.109375
0.06640625
0.123046875
0.06640625
0.095703125
0.08203125
0.130859375
0.078125
0.125
0.109375
0.1328125
0.09375
0.1484375
0.0703125
0.15625
0.08203125
0.16015625
0.134765625
0.12109375
0.1328125
0.1171875
0.103515625
0.10546875
0.083984375
0.099609375
0.07421875
0.087890625
0.07421875
0.099609375
0.078125
0.080078125
0.083984375
0.076171875
0.080078125
0.0625
0.09375
0.044921875
0.087890625
0.0703125
0.087890625
0.126953125
0.060546875
0.12890625
0.0703125
0.119140625


0.1015625
0.09375
0.095703125
0.08203125
0.1640625
0.07421875
0.142578125
0.0859375
0.125
0.09765625
0.12109375
0.091796875
0.109375
0.10546875
0.15625
0.08984375
0.134765625
0.091796875
0.0703125
0.125
0.12890625
0.103515625
0.12890625
0.056640625
0.16015625
0.0859375
0.142578125
0.13671875
0.15625
0.134765625
0.15234375
0.111328125
0.109375
0.12109375
0.099609375
0.138671875
0.13671875
0.12109375
0.111328125
0.0703125
0.134765625
0.0703125
0.138671875
0.09765625
0.154296875
0.0859375
0.13671875
0.0859375
0.138671875
0.111328125
0.154296875
0.123046875
0.111328125
0.109375
0.0703125
0.1328125
0.0625
0.099609375
0.083984375
0.09375
0.0703125
0.068359375
0.08984375
0.0703125
0.064453125
0.068359375
0.07421875
0.05078125
0.08984375
0.087890625
0.05078125
0.072265625
0.072265625
0.04296875
0.103515625
0.072265625
0.119140625
0.07421875
0.08984375
0.044921875
0.12890625
0.095703125
0.08203125
0.111328125
0.083984375
0.115234375
0.099609375
0.125
0.072265625
0.111328125
0.095703125
0.107421

In [ ]:
train() #symmetric NN

0.54296875
0.560546875
0.4453125
0.30078125
0.287109375
0.46484375
0.470703125
0.44921875
0.359375
0.3828125
0.421875
0.3671875
0.306640625
0.236328125
0.5078125
0.52734375
0.2890625
0.255859375
0.404296875
0.3515625
0.376953125
0.28125
0.31640625
0.443359375
0.349609375
0.2109375
0.28125
0.427734375
0.33984375
0.224609375
0.28515625
0.34375
0.24609375
0.234375
0.23046875
0.3203125
0.265625
0.21484375
0.283203125
0.19140625
0.25
0.28515625
0.255859375
0.197265625
0.25
0.275390625
0.18359375
0.375
0.275390625
0.177734375
0.326171875
0.23046875
0.166015625
0.26953125
0.267578125
0.185546875
0.25390625
0.26953125
0.234375
0.208984375
0.21484375
0.177734375
0.203125
0.2109375
0.232421875
0.15234375
0.201171875
0.2109375
0.1484375
0.201171875
0.154296875
0.134765625
0.123046875
0.14453125
0.166015625
0.15625
0.146484375
0.12890625
0.23828125
0.1484375
0.17578125
0.138671875
0.1640625
0.1171875
0.1171875
0.19140625
0.25
0.095703125
0.134765625
0.306640625
0.138671875
0.201171875
0.1796875
0.

0.123046875
0.080078125
0.07421875
0.0703125
0.087890625
0.099609375
0.0859375
0.06640625
0.08203125
0.0703125
0.09765625
0.119140625
0.091796875
0.126953125
0.111328125
0.0859375
0.115234375
0.11328125
0.119140625
0.11328125
0.107421875
0.15234375
0.123046875
0.134765625
0.109375
0.1015625
0.166015625
0.068359375
0.18359375
0.17578125
0.154296875
0.171875
0.16015625
0.2421875
0.166015625
0.193359375
0.138671875
0.138671875
0.140625
0.16015625
0.208984375
0.080078125
0.16796875
0.09765625
0.162109375
0.10546875
0.1875
0.123046875
0.115234375
0.15234375
0.154296875
0.19140625
0.166015625
0.16796875
0.150390625
0.111328125
0.119140625
0.1640625
0.138671875
0.220703125
0.19921875
0.1640625
0.158203125
0.12890625
0.173828125
0.11328125
0.138671875
0.095703125
0.15625
0.107421875
0.115234375
0.109375
0.1328125
0.126953125
0.06640625
0.13671875
0.07421875
0.107421875
0.0390625
0.119140625
0.05859375
0.142578125
0.08984375
0.11328125
0.05078125
0.107421875
0.091796875
0.16796875
0.05859375
0.

0.083984375
0.10546875
0.06640625
0.08203125
0.083984375
0.076171875
0.111328125
0.05078125
0.1328125
0.08984375
0.103515625
0.1015625
0.123046875
0.091796875
0.123046875
0.076171875
0.09765625
0.169921875
0.09375
0.11328125
0.107421875
0.166015625
0.107421875
0.080078125
0.09375
0.111328125
0.072265625
0.14453125
0.05859375
0.15625
0.08203125
0.119140625
0.08984375
0.103515625
0.091796875
0.076171875
0.076171875
0.09765625
0.0625
0.09375
0.0703125
0.103515625
0.0625
0.08203125
0.056640625
0.1015625
0.099609375
0.09375
0.15625
0.115234375
0.10546875
0.095703125
0.10546875
0.087890625
0.103515625
0.0703125
0.08984375
0.068359375
0.109375
0.09375
0.05859375
0.103515625
0.056640625
0.072265625
0.080078125
0.064453125
0.08984375
0.083984375
0.05859375
0.06640625
0.052734375
0.091796875
0.083984375
0.068359375
0.10546875
0.0625
0.11328125
0.056640625
0.126953125
0.091796875
0.12890625
0.09375
0.07421875
0.123046875
0.080078125
0.095703125
0.09375
0.1015625
0.0703125
0.0703125
0.056640625
0.

0.068359375
0.076171875
0.056640625
0.0546875
0.078125
0.064453125
0.0703125
0.06640625
0.076171875
0.052734375
0.07421875
0.044921875
0.06640625
0.076171875
0.068359375
0.095703125
0.080078125
0.052734375
0.087890625
0.0703125
0.0703125
0.119140625
0.08984375
0.12109375
0.111328125
0.07421875
0.0859375
0.083984375
0.10546875
0.0859375
0.107421875
0.09375
0.0859375
0.080078125
0.1015625
0.08203125
0.13671875
0.0703125
0.123046875
0.1484375
0.103515625
0.16796875
0.10546875
0.177734375
0.20703125
0.0703125
0.16015625
0.134765625
0.146484375
0.203125
0.0703125
0.1328125
0.08203125
0.119140625
0.130859375
0.109375
0.1328125
0.115234375
0.115234375
0.0625
0.138671875
0.09765625
0.080078125
0.126953125
0.076171875
0.095703125
0.109375
0.13671875
0.125
0.0546875
0.1328125
0.091796875
0.125
0.111328125
0.08984375
0.09375
0.078125
0.08984375
0.076171875
0.0859375
0.083984375
0.10546875
0.076171875
0.083984375
0.083984375
0.099609375
0.140625
0.134765625
0.109375
0.078125
0.119140625
0.07226562